In [300]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
import PIL
import math
from tqdm import tqdm
from torch import nn
from torch.nn import functional as F

In [301]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
device

device(type='cpu')

In [302]:
white = 255
black = 0

def img_tranform(fill_color):
    return transforms.Compose([
        transforms.RandomRotation(360, fill=fill_color),

        transforms.CenterCrop(300),
        
        transforms.ColorJitter(brightness=(0.8, 1), contrast=(0.45, 1)),

    ])

def fix_background_color(img):
    colors = sorted(img.getcolors(), key=lambda pair: pair[0], reverse=True)
    replace_color = colors[0][1]
    remove_color = colors[2][1] if colors[2][1] > colors[1][1] else colors[1][1]

    data = np.array(img)
    data[data == remove_color] = replace_color
    return PIL.Image.fromarray(data)

operators_transform = transforms.Compose([
    
    transforms.Grayscale(num_output_channels=1),
    # Randomly scale up and down
    transforms.RandomAffine(0, scale=(0.9, 1.1), fillcolor=white),
    img_tranform(white),
    
    transforms.Lambda(fix_background_color),
    transforms.ToTensor(),
])

minst_transform = transforms.Compose([
    # Scale up
    transforms.Pad(70),
    transforms.RandomAffine(0, scale=(2.5, 3.2), fillcolor=0),
    
    img_tranform(black),
    transforms.ToTensor(),
])

In [303]:
# Operators Dataset
operators_dataset = datasets.ImageFolder(root='operators', transform=operators_transform)
# operators_dataset.transform(operators_dataset[0][0])

In [304]:
# Minst Dataset
minst_dataset = datasets.MNIST("", transform=minst_transform, download=True)
# minst_dataset.transform(minst_dataset[0][0])

In [305]:
len(operators_dataset)

5

In [306]:
class FusionDataset(Dataset):
    """Custom Dataset for loading CelebA face images"""

    def __init__(self, operators_dataset, minst_dataset):
        self.operators_dataset = operators_dataset
        self.minst_dataset = minst_dataset

    def __getitem__(self, index):
        if index < len(self.operators_dataset):
            return self.operators_dataset[index]
        else:
            return self.minst_dataset[index - len(self.operators_dataset)]

    def __len__(self):
        return len(self.operators_dataset) + len(self.minst_dataset)

In [307]:
dataset = FusionDataset(operators_dataset, minst_dataset)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=4,
                                         shuffle=False, num_workers=2)

In [308]:
# Creating a Net class object, which consists of 2 convolutional layers, max-pool layers and fully-connected layers
class Conv_Net(nn.Module):
    def __init__(self, nb_hidden=50):        
        super(Conv_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3)  # the first convolutional layer, which processes the input image
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)  # the second convolutional layer, which gets the max-pooled set
        self.fc1 = nn.Linear(800, nb_hidden)  # the first fully-connected layer, which gets flattened max-pooled set
        self.fc2 = nn.Linear(nb_hidden, 10)  # the second fully-connected layer that outputs the result

    # Creating the forward pass
    def forward(self, x):
        
        # The first two layers
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2))
        
        # The second two layers
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2)) 
        
        # Flattening the data set for fully-connected layer
        x = x.view(x.size(0), -1)
    
        # The first fully-connected layer
        x = F.relu(self.fc1(x))
        
        # The second full-connected layer
        x = self.fc2(x)
        
        return x

In [309]:
model = Conv_Net()

In [319]:
losses = []



# Defining the optimizer for GD
lr = 0.2
optimizer = torch.optim.Adam(model.parameters(), lr = lr) 

# Defining the criterion to calculate loss
criterion = nn.CrossEntropyLoss()

offset = int(len(train_loader) * 0.2)
print(0)
test, train = torch.Tensor(list(train_loader)[:offset]).view(-1, 784), torch.Tensor(list(train_loader)[offset:]).view(-1, 784)
print(1)
# Learning loop
nb_epochs = 10
mini_batch_size = 100
for e in tqdm(range(nb_epochs)):
    # Train the input dataset by dividing it into mini_batch_size small datasets
    for train_input, train_target in train.split(mini_batch_size):

        # Model computations
        output = model(train_input)
        loss = criterion(output, train_target) 
        print("train", loss)
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()
        
    losses.append(loss)
    
    # Train the input dataset by dividing it into mini_batch_size small datasets
    for test_input, test_target in test.split(mini_batch_size):
        
        output = model(test_input)
        loss = criterion(output, test_target) 
        print("test", loss)
        
    # Validation
#     with torch.set_grad_enabled(False):
#         for local_batch, local_labels in validation_generator:
#             # Transfer to GPU
#             local_batch, local_labels = local_batch.to(device), local_labels.to(device)

#             # Model computations
#             [...]

0


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x46a9f8a58>>
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x46a9f8a58>>
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/henrydeclety/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 962, in __del__
  File "/Users/henrydeclety/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 962, in __del__
    self._shutdown_workers()
    self._shutdown_workers()
  File "/Users/henrydeclety/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 942, in _shutdown_workers
  File "/Users/henrydeclety/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 942, in _shutdown_workers
    w.join()
    w.join()
  File "/anaconda3/lib/python3.6/m

ValueError: only one element tensors can be converted to Python scalars

In [281]:
def train_model(model, train_inputs, train_targets, mini_batch_size=100, remember_losses=True, lr=1e-3, nb_epochs=20):
    
    """
        Train the PyTorch model on the training set.
        
        Parameters
        ----------
        model : PyTorch NN object
            PyTorch neural network model      
        train_inputs : torch.Tensor object
            The input train feature set
        train_targets : torch.Tensor object
            The input train label set
        mini_batch_size : int
            The size of the batch processing size
        remember_losses : boolean
            True if remember losses for model evaluation, False if not
        lr : float
            Learning rate for the model training
        nb_epochs : int
            The number of epochs used to train the model
            
        Returns
        -------
        
        NN object or (NN object, list)
            If remember_losses is True then the function returns both
            the trained model and the list of losses for each epoch. 
            If remember_losses is False then the function return only
            the trained model.
    """
    
    # Defining the learning rate and number of epochs
    losses = []

    # Defining the optimizer for GD
    optimizer = torch.optim.Adam(model.parameters(), lr = lr) 
    
    # Defining the criterion to calculate loss
    criterion = nn.CrossEntropyLoss()
    
    # Learning loop
    for e in range(nb_epochs):
        # Train the input dataset by dividing it into mini_batch_size small datasets
        for train_input, train_target in zip(train_inputs.split(mini_batch_size), train_targets.split(mini_batch_size)):
            output = model(train_input)
            loss = criterion(output, train_target) 
            optimizer.zero_grad() 
            loss.backward()
            optimizer.step()
        losses.append(loss)
        print('%dth epoch is finished and the loss is %f' % (e+1, loss))
           
    
    return model, losses